In [2]:
import pandas as pd
import random
import numpy as np
from random import randint

import torch
from transformers import AutoTokenizer, AutoModel

import time

import memory_profiler
%load_ext memory_profiler

from pathlib import Path

In [2]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [3]:
%load_ext autoreload
%autoreload 2

from pubmed_landscape_src.data import generate_embeddings

In [4]:
variables_path = Path("../results/variables")
figures_path = Path("../results/figures")
berenslab_data_path = Path("/gpfs01/berens/data/data/pubmed_processed")

# Import dataframe

In [3]:
%%time
%%memit
# import clean_df_updated
clean_df_updated_reparsed_filtered_with_authors_ISSN=pd.read_pickle(variables_path / "clean_df_updated_reparsed_filtered_with_authors_ISSN")

tcmalloc: large alloc 1073741824 bytes == 0x8c4e18000 @ 
tcmalloc: large alloc 1233903616 bytes == 0x96edbe000 @ 


peak memory: 49300.59 MiB, increment: 48788.76 MiB
CPU times: user 40.7 s, sys: 43.3 s, total: 1min 24s
Wall time: 1min 24s


In [4]:
# extract abstract texts
abstracts = clean_df_updated_reparsed_filtered_with_authors_ISSN['AbstractText'].tolist()

In [6]:
clean_df_updated_reparsed_filtered_with_authors_ISSN.head()

,PMID,Title,AbstractText,Language,Journal,Date,filename
0,1133452,Reflections on the special senses in relation ...,"In congenital blindness, the absence of vision...",eng,Journal of the American Psychoanalytic Associa...,1975,pubmed21n0038.xml
1,1133453,Tracing a memory.,Comparisons of the nests of associations withi...,eng,Journal of the American Psychoanalytic Associa...,1975,pubmed21n0038.xml
2,1133454,Some pitfalls in the assessment of analyzabili...,We have described some of the problems encount...,eng,Journal of the American Psychoanalytic Associa...,1975,pubmed21n0038.xml
3,1133455,"Variability of the iron, copper and mercury co...","The relative iron, copper and mercury contents...",eng,The journal of histochemistry and cytochemistr...,1975 May,pubmed21n0038.xml
4,1133526,"Hypophysectomy of the tammar wallaby, Macropus...",A technique of hypophysectomy and regimes of p...,eng,The Journal of endocrinology,1975 Mar,pubmed21n0038.xml


# Obtaining the PubMedBERT embeddings of the abstracts

## Model specification

In [7]:
%%memit
# specifying model 
checkpoint = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
#"dmis-lab/biobert-v1.1"

#"dmis-lab/biobert-v1.1"
#"allenai/scibert_scivocab_uncased"
#"bert-base-uncased"
#"dmis-lab/biobert-v1.1"
#"allenai/scibert_scivocab_uncased"
#"microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
#"dmis-lab/biobert-base-cased-v1.2"
#"bert-base-uncased"
#"allenai/scibert_scivocab_uncased"
#"bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


peak memory: 49308.05 MiB, increment: 8.70 MiB


In [8]:
# specifying that calculations are performed on gpu
device = "cuda" if torch.cuda.is_available() else "cpu" 
print("running on device: {}".format(device))

running on device: cuda


### Batch 1
Batches of 5M

In [10]:
%%time
%%memit

mat = np.empty([5000000, 768])
abstract_batch=abstracts[:5000000]
for i, abst in enumerate(abstract_batch): 
    _, mat[i], _ = generate_embeddings(abst, tokenizer, model, device)
    last_iter=np.array([i])
    np.save('variables/last_iter_batch_1',last_iter)

# save embedding
np.save(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_1', mat) 

tcmalloc: large alloc 30720000000 bytes == 0x7fabdcf20000 @ 


peak memory: 78482.31 MiB, increment: 28828.03 MiB
CPU times: user 15h 22min 27s, sys: 3h 47min 20s, total: 19h 9min 47s
Wall time: 19h 22min 41s


### Batch 2

In [21]:
%%time
%%memit


abstract_batch=abstracts[5000000:10000000]
mat = np.empty([len(abstract_batch), 768])
for i, abst in enumerate(abstract_batch): 
    _, mat[i], _ = generate_embeddings(abst, tokenizer, model, device)
    last_iter=np.array([i+5000000])
    np.save('variables/last_iter_batch_2',last_iter)
    
# save embedding
np.save(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_2', mat)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tcmalloc: large alloc 30720000000 bytes == 0x7fa48af20000 @ 


peak memory: 80344.27 MiB, increment: 30934.36 MiB
CPU times: user 15h 10min 6s, sys: 4h 5min 1s, total: 19h 15min 7s
Wall time: 19h 27min 59s


### Batch 3

In [ ]:
%%time
%%memit

mat = np.empty([5000000, 768])
abstract_batch=abstracts[10000000:15000000]
for i, abst in enumerate(abstract_batch): 
    _, mat[i], _ = generate_embeddings(abst, tokenizer, model, device)
    last_iter=np.array([i+10000000])
    np.save('variables/last_iter_batch_3',last_iter)
    
# save embedding
np.save(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_3', mat)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tcmalloc: large alloc 30720000000 bytes == 0x7f8e17e3a000 @ 


### Batch 4

In [ ]:
%%time
%%memit

mat = np.empty([5000000, 768])
abstract_batch=abstracts[15000000:20000000]
for i, abst in enumerate(abstract_batch): 
    _, mat[i], _ = generate_embeddings(abst, tokenizer, model, device)
    last_iter=np.array([i+15000000])
    np.save('variables/last_iter_batch_4',last_iter)
    
# save embedding
np.save(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_4', mat)

### Batch 5

In [ ]:
%%time
%%memit

abstract_batch=abstracts[20000000:]
mat = np.empty([len(abstract_batch), 768])
for i, abst in enumerate(abstract_batch): 
    _, mat[i], _ = generate_embeddings(abst, tokenizer, model, device)
    last_iter=np.array([i+20000000])
    np.save('variables/last_iter_batch_5',last_iter)
    
# save embedding
np.save(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_5', mat)

# Concatenate the batches

In [30]:
embeddings_reparsed_batch_1 = np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_1.npy')

tcmalloc: large alloc 30720000000 bytes == 0x7fa48af20000 @ 


In [31]:
embeddings_reparsed_batch_2 = np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_2.npy')

tcmalloc: large alloc 30720000000 bytes == 0x7f8e17e3a000 @ 


In [32]:
embeddings_reparsed_batch_3 = np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_3.npy')

tcmalloc: large alloc 30720000000 bytes == 0x7f953ef1a000 @ 


In [33]:
embeddings_reparsed_batch_4 = np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_4.npy')

tcmalloc: large alloc 30720000000 bytes == 0x7f9c65ffa000 @ 


In [34]:
#embeddings_reparsed_batch_5 = np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_batch_5.npy')
embeddings_reparsed_batch_5 = mat

In [35]:
%%time
embeddings_reparsed_all=np.vstack((embeddings_reparsed_batch_1, embeddings_reparsed_batch_2, embeddings_reparsed_batch_3, embeddings_reparsed_batch_4, embeddings_reparsed_batch_5))
embeddings_reparsed_all.shape

tcmalloc: large alloc 127101853696 bytes == 0x7f70781b4000 @ 


CPU times: user 22.5 s, sys: 30.5 s, total: 53 s
Wall time: 53 s


(20687150, 768)